In [17]:
from sklearn.datasets import load_iris
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from qiskit.circuit.library import ZFeatureMap, RealAmplitudes
from qiskit_machine_learning.algorithms import VQC
from qiskit_algorithms.optimizers import COBYLA
from qiskit_machine_learning.utils import algorithm_globals
import numpy as np
import matplotlib.pyplot as plt
import time

# Seed
algorithm_globals.random_seed = 42
np.random.seed(42)

# --- 1. Persiapan Data (LDA + 2 Dimensi) ---
iris = load_iris()
X = iris.data
y = iris.target

print("1. Mengompres data menggunakan LDA...")
lda = LinearDiscriminantAnalysis(n_components=2)
X_lda = lda.fit_transform(X, y)

scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X_lda)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# --- 2. Setup Komponen QML (Slim & Fit) ---
num_qubits = 2
feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=2)
ansatz = RealAmplitudes(num_qubits=num_qubits, reps=3, entanglement='full')
optimizer = COBYLA(maxiter=150) # Cukup 150 karena tugasnya binary (Yes/No)

# --- 3. TRAINING MANUAL ONE-VS-REST ---
print("\n--- Mulai Training 3 Model Terpisah (Manual OvR) ---")
classifiers = {} # Tempat menyimpan 3 model kita
target_classes = [0, 1, 2] # Kelas yang mau kita pelajari

start_time = time.time()

for cls in target_classes:
    print(f"\nTraining Detektor Kelas {cls}...")
    
    # KUNCI RAHASIA DI SINI:
    # Kita ubah labelnya. 
    # Jika label asli == kelas target, jadi 1 (Yes).
    # Jika label asli != kelas target, jadi 0 (No).
    y_train_binary = np.where(y_train == cls, 1, 0)
    
    # Buat VQC baru untuk setiap kelas
    vqc = VQC(feature_map=feature_map,
              ansatz=ansatz,
              optimizer=optimizer)
    
    # Train dengan label biner (0 atau 1)
    vqc.fit(X_train, y_train_binary)
    
    # Simpan model yang sudah pintar
    classifiers[cls] = vqc

end_time = time.time()
print(f"\nWaktu Total Training: {end_time - start_time:.2f} detik")

# --- 4. PREDIKSI MANUAL ---
print("\n--- Melakukan Prediksi (Voting) ---")

# Kita ambil 15 data test pertama
sample_data = X_test[:15]
sample_labels = y_test[:15]
final_predictions = []

for data_point in sample_data:
    # Siapkan skor untuk [Kelas 0, Kelas 1, Kelas 2]
    # Siapa yang jawab "YES" (1), dia dapat poin.
    scores = []
    
    # Data harus dalam bentuk list of list [[x1, x2]]
    data_reshaped = data_point.reshape(1, -1)
    
    for cls in target_classes:
        # Tanya setiap model
        pred = classifiers[cls].predict(data_reshaped)
        # Pred hasilnya scalar atau array 0-dimensi
        scores.append(int(pred)) 
    
    # Logika Voting Sederhana:
    # scores isinya misal [0, 1, 0] -> Artinya Model 1 bilang YES. Maka prediksi = 1.
    # scores isinya misal [0, 0, 1] -> Artinya Model 2 bilang YES. Maka prediksi = 2.
    
    if sum(scores) == 1:
        # Jika cuma 1 model yang bilang YES, ambil index-nya
        final_predictions.append(np.argmax(scores))
    elif sum(scores) > 1:
        # Jika 2 model ngaku-ngaku (konflik), kita ambil yang pertama (atau bisa pakai probabilitas nanti)
        final_predictions.append(np.argmax(scores)) 
    else:
        # Jika tidak ada yang ngaku (semua 0), kita tebak kelas 0 atau biarkan error
        final_predictions.append(0) # Fallback

# --- 5. Evaluasi ---
print(f"Prediksi Manual : {final_predictions}")
print(f"Label Asli      : {sample_labels}")

# Hitung Akurasi Sampel
correct = np.sum(np.array(final_predictions) == sample_labels)
acc = correct / len(sample_labels)
print(f"Akurasi Sampel  : {acc*100:.0f}%")


/tmp/ipykernel_30760/2690910076.py:33: DeprecationWarning: The class ``qiskit.circuit.library.data_preparation._z_feature_map.ZFeatureMap`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the z_feature_map function as a replacement. Note that this will no longer return a BlueprintCircuit, but just a plain QuantumCircuit.
  feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=2)
/tmp/ipykernel_30760/2690910076.py:34: DeprecationWarning: The class ``qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.real_amplitudes instead.
  ansatz = RealAmplitudes(num_qubits=num_qubits, reps=3, entanglement='full')
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


1. Mengompres data menggunakan LDA...

--- Mulai Training 3 Model Terpisah (Manual OvR) ---

Training Detektor Kelas 0...


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.



Training Detektor Kelas 1...


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.



Training Detektor Kelas 2...

Waktu Total Training: 85.11 detik

--- Melakukan Prediksi (Voting) ---
Prediksi Manual : [np.int64(1), np.int64(0), np.int64(1), 0, np.int64(1), np.int64(0), 0, np.int64(2), np.int64(1), np.int64(1), np.int64(2), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Label Asli      : [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0]
Akurasi Sampel  : 80%
